# Linear Classification

In this lab you will implement parts of a linear classification model using the regularized empirical risk minimization principle. By completing this lab and analysing the code, you gain deeper understanding of these type of models, and of gradient descent.


## Problem Setting

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal (1) and abnormal (0). The training data contains 80 SPECT images from which 22 binary features have been extracted. The goal is to predict the label for an unseen test set of 187 tomography images.

In [1]:
import urllib
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

testfile = urllib.request.URLopener()
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.train", "SPECT.train")
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.test", "SPECT.test")

df_train = pd.read_csv('SPECT.train',header=None)
df_test = pd.read_csv('SPECT.test',header=None)

train = df_train.as_matrix()
test = df_test.as_matrix()

y_train = train[:,0]
X_train = train[:,1:]
y_test = test[:,0]
X_test = test[:,1:]

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


### Exercise 1

Analyze the function learn_reg_ERM(X,y,lambda) which for a given $n\times m$ data matrix $\textbf{X}$ and binary class label $\textbf{y}$ learns and returns a linear model $\textbf{w}$.
The binary class label has to be transformed so that its range is $\left \{-1,1 \right \}$. 
The trade-off parameter between the empirical loss and the regularizer is given by $\lambda > 0$. 
Try to understand each step of the learning algorithm and comment each line.

In [2]:
def learn_reg_ERM(X,y,lbda):
    max_iter = 200
    e  = 0.001
    alpha = 1.
    
    # initial random beta values
    
    w = np.random.randn(X.shape[1]);
    for k in np.arange(max_iter):
        
        # Decision function : product of x and beta values 
        h = np.dot(X,w)
        
        # Finding hinge loss and gradient between predicted and actual
        l,lg = loss(h, y)
#         print ('loss: {}'.format(np.mean(l)))
        
        # Computing L2 -regularizer and the gradient of the regularizer function at point  𝐰
        r,rg = reg(w, lbda)
        
        # Applying  gradient with X value and adjusting with regularizer paramter
        g = np.dot(X.T,lg) + rg 
        
        # Determining step size
        if (k > 0):
            alpha = alpha * (np.dot(g_old.T,g_old))/(np.dot((g_old - g).T,g_old))
        
        # reforming the weights based on gradient and step value
        w = w - alpha * g
        
        # Comparing distance 1 norm distance , if it is lower than e measurement, model is ready
        if (np.linalg.norm(alpha * g) < e):
            break
            
        # preserving the current gradient value for next iteration
        g_old = g
        
    return w

### Exercise 2

Fill in the code for the function loss(h,y) which computes the hinge loss and its gradient. 
This function takes a given vector $\textbf{y}$ with the true labels $\in \left \{-1,1\right \}$ and a vector $\textbf{h}$ with the function values of the linear model as inputs. The function returns a vector $\textbf{l}$ with the hinge loss $\max(0, 1 − y_{i} h_{i})$ and a vector $\textbf{g}$ with the gradients of the hinge loss at the points $h_i$. 

The partial derivative of the hinge loss $h_i$ with respect to the $i$-th position of the weight vector $\textbf{w}$ is $g_{i} = −y x_{i}$ if $l_{i} > 0$, else $g_{i} = 0$).

In [123]:
def loss(h, y):
    ##################
    #INSERT CODE HERE#
    ##################
    
    g=[0]*h.shape[0]
    expr = 1-np.multiply(h, y)
    l= np.maximum(0, expr)
    
    for i in range(h.shape[0]) :
        if l[i] > 0:
#             g[i]=np.dot(-1*y,X[:,i])
            g[i]=sum(-1*y*h[10])
    g=np.array(g)
    return l, l

In [1]:
def loss(h, y):
    ##################
    #INSERT CODE HERE#
    ##################
    y_index = y<=0
    y[y_index] = -1
    loss = []
    grad = []
    for i in range(len(y)):
        li = 0 if 1-y[i]*h[i]<0 else 1-y[i]*h[i]
        loss.append(li)
        grad.append(-y[i]*h[i] if li>0 else 0)
    
    l = np.array(loss)
    g = np.array(grad)
    return l, g

In [2]:
def loss(h, y):
    ##################
    #INSERT CODE HERE#
    ##################
    l=np.maximum(0,1-y*h)
    g=-y*(l>0)
    
    return l, g

### Exercise 3

Fill in the code for the function reg(w,lambda) which computes the $\mathcal{L}_2$-regularizer and the gradient of the regularizer function at point $\textbf{w}$. 


$$r = \frac{\lambda}{2} \textbf{w}^{T}\textbf{w}$$

$$g = \lambda \textbf{w}$$

In [17]:
def reg(w, lbda):
    ##################
    #INSERT CODE HERE#
    ##################
    r=(lbda/2)*w.T*w
    g=lbda*w
    return r, g

### Exercise 4

Fill in the code for the function predict(w,x) which predicts the class label $y$ for a data point $\textbf{x}$ or a matrix $X$ of data points (row-wise) for a previously trained linear model $\textbf{w}$. If there is only a data point given, the function is supposed to return a scalar value. If a matrix is given a vector of predictions is supposed to be returned.

In [65]:
def predict(w, X):
    ##################
    #INSERT CODE HERE#
    ##################
    
    #preds=[0]*X.shape[0]
    #for j in range(X.shape[0]):
    #    for  i in range(X.shape[1]):
    #        preds[j]+=-1*X[j,i]*w[i]
    
    preds=np.dot(X,w)
    
    p=np.array([ 0 if x < 0 else 1 for x in preds])
    
    return p

### Exercise 5

#### 5.1 
Train a linear model on the training data and classify all 187 test instances afterwards using the function predict. 
Please note that the given class labels are in the range $\left \{0,1 \right \}$, however the learning algorithm expects a label in the range of $\left \{-1,1 \right \}$. Then, compute the accuracy of your trained linear model on both the training and the test data. 

In [58]:
def converts(data, to=-1, frm=0):
    nm=data
    try :
        for i in range(data.shape[0]):
            for j in range(data.shape[1]):
                if(data[i,j]==frm):
                    nm[i,j]=to
                
    except IndexError as error: 
        nm=np.array([to if x==frm else 1 for x in data])
    
    return nm

In [63]:
##################
#INSERT CODE HERE#
##################
from scipy import stats
X_train_std=converts(X_train)
y_train_std=converts(y_train)
w=learn_reg_ERM(X_train_std,y_train_std,0.8)
preds_train=predict(w,X_train_std)

X_test_std=converts(X_test) 
y_test_std=converts(y_test)
preds_test=predict(w,X_test_std)


#### 5.2
Compare the accuracy of the linear model with the accuracy of a random forest and a decision tree on the training and test data set.

In [64]:
##################
#INSERT CODE HERE#
##################

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

print('Linear classification')

print('\nR^2 Value')
print("Train : "+str(r2_score(y_train, preds_train)))
print("Test : "+str(r2_score(y_test, preds_test)))

print('\nMSE Value')
print("Train : "+str(accuracy_score(y_train,preds_train)))
print("Test : "+str(accuracy_score(y_test,preds_test)))


clf = RandomForestClassifier()
rd=clf.fit(X_train,y_train)
preds_train_rf=clf.predict(X_train)
preds_test_rf=clf.predict(X_test)


print('\nRandom Forest classification')
print('\nR^2 Value')
print("Train : "+str(r2_score(y_train, preds_train_rf)))
print("Test : "+str(r2_score(y_test, preds_test_rf)))
print('\nAccuracy Value')
print("Train : "+str(accuracy_score(y_train,preds_train_rf)))
print("Test : "+str(accuracy_score(y_test,preds_test_rf)))

Linear classification

R^2 Value
Train : 0.30000000000000004
Test : -4.943410852713178

MSE Value
Train : 0.825
Test : 0.5614973262032086

Random Forest classification

R^2 Value
Train : 0.65
Test : -3.0589147286821703
\Accuacy Value
Train : 0.9125
Test : 0.7005347593582888


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
